In [3]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/data-gec/whole_data.csv


In [4]:
import pandas as pd
import tensorflow as tf
import numpy as np

In [5]:
df = pd.read_csv('/kaggle/input/data-gec/whole_data.csv', dtype=str)
df = df.dropna()

In [6]:
df = df.drop('correct_input', axis=1)
df = df.drop('correct_output', axis = 1)

In [7]:
df.head()

,correct,incorrect
0,And he took in my favorite subjects like soccer .,And he took in my favorite subject like soccer .
1,"Actually , he was the one who let me know abou...","Actually , who let me know about Lang was him ."
2,His Kanji ability is much better than mine .,His Kanji's ability is much better than me .
3,We have known each other for only half a year ...,We have known each other for only half a year ...
4,I heard a sentence last night when I was watch...,I heard a sentence last night when I watched TV .


In [8]:
df = df.rename(columns={'correct': 'target', 'incorrect': 'input'})

In [9]:
df.head()

,target,input
0,And he took in my favorite subjects like soccer .,And he took in my favorite subject like soccer .
1,"Actually , he was the one who let me know abou...","Actually , who let me know about Lang was him ."
2,His Kanji ability is much better than mine .,His Kanji's ability is much better than me .
3,We have known each other for only half a year ...,We have known each other for only half a year ...
4,I heard a sentence last night when I was watch...,I heard a sentence last night when I watched TV .


In [10]:
!pip install -q transformers

In [11]:
from transformers import T5Tokenizer, T5ForConditionalGeneration, AutoTokenizer

In [12]:
t5_model = T5ForConditionalGeneration.from_pretrained('anujraymajhi/t5-GEC-128len-6e')
tokenizer = AutoTokenizer.from_pretrained('t5-base')

/opt/conda/lib/python3.7/site-packages/transformers/models/t5/tokenization_t5_fast.py:165: FutureWarning: This tokenizer was incorrectly instantiated with a model max length of 512 which will be corrected in Transformers v5.
For now, this behavior is kept to avoid breaking backwards compatibility when padding/encoding with `truncation is True`.
- Be aware that you SHOULD NOT rely on t5-base automatically truncating your input to 512 when padding/encoding.
- If you want to encode/pad to sequences longer than 512 you can either instantiate this tokenizer with `model_max_length` or pass `max_length` when encoding/padding.
- To avoid this warning, please instantiate this tokenizer with `model_max_length` set to your preferred value.
  FutureWarning,


In [13]:
!pip install -q datasets

In [14]:
from datasets import Dataset

df_test = pd.read_csv("/kaggle/input/jflegtest/eval_total_dataset.csv", dtype=str)
train_dataset = Dataset.from_pandas(df)
test_dataset = Dataset.from_pandas(df_test)

In [15]:
from torch.utils.data import Dataset, DataLoader

In [16]:
class GrammarDataset(Dataset):
    def __init__(self, dataset, tokenizer,print_text=False):         
        self.dataset = dataset
        self.pad_to_max_length = False
        self.tokenizer = tokenizer
        self.print_text = print_text
        self.max_len = 128
  
    def __len__(self):
        return len(self.dataset)


    def tokenize_data(self, example):
        input_, target_ = example['input'], example['target']

        # tokenize inputs
        tokenized_inputs = tokenizer(input_, pad_to_max_length=self.pad_to_max_length, 
                                            max_length=self.max_len,
                                            return_attention_mask=True, truncation = True)
    
        tokenized_targets = tokenizer(target_, pad_to_max_length=self.pad_to_max_length, 
                                            max_length=self.max_len,
                                            return_attention_mask=True, truncation= True)

        inputs={"input_ids": tokenized_inputs['input_ids'],
            "attention_mask": tokenized_inputs['attention_mask'],
            "labels": tokenized_targets['input_ids']
        }
        
        return inputs

  
    def __getitem__(self, index):
        inputs = self.tokenize_data(self.dataset[index])
        
        if self.print_text:
            for k in inputs.keys():
                print(k, len(inputs[k]))

        return inputs

In [ ]:
from datasets import load_metric
rouge_metric = load_metric("rouge")

In [17]:
train_dataset

Dataset({
    features: ['target', 'input', '__index_level_0__'],
    num_rows: 496295
})

In [18]:
from transformers import (
    T5ForConditionalGeneration, T5Tokenizer, 
    Seq2SeqTrainingArguments, Seq2SeqTrainer, DataCollatorForSeq2Seq
  )

In [19]:
data_collator = DataCollatorForSeq2Seq(tokenizer, model=t5_model, padding='longest', return_tensors='pt')

In [21]:
batch_size = 32
args = Seq2SeqTrainingArguments(output_dir="kaggle/working/weights",
                        evaluation_strategy="steps",
                        per_device_train_batch_size=batch_size,
                        per_device_eval_batch_size=batch_size,
                        learning_rate=2e-5,
                        num_train_epochs=3,
                        weight_decay=0.01,
                        save_total_limit=2,
                        predict_with_generate=True,
                        fp16 = True,
                        gradient_accumulation_steps = 6,
                        eval_steps = 625,
                        save_steps = 2500,
                        load_best_model_at_end=True,
                        logging_dir="/logs")

In [22]:
trainer = Seq2SeqTrainer(model=t5_model, 
                args=args, 
                train_dataset= GrammarDataset(train_dataset, tokenizer),
                eval_dataset=GrammarDataset(test_dataset, tokenizer),
                tokenizer=tokenizer,
                data_collator=data_collator)

Using cuda_amp half precision backend


In [23]:
os.environ["WANDB_DISABLED"] = "true"

In [ ]:
trainer.train()

In [ ]:
df.head()

In [29]:
trainer.training_step()

TypeError: training_step() missing 2 required positional arguments: 'model' and 'inputs'

In [25]:
trainer.evaluate()

***** Running Evaluation *****
  Num examples = 800
  Batch size = 64


{'eval_loss': 0.3520869314670563,
 'eval_runtime': 3.4078,
 'eval_samples_per_second': 234.753,
 'eval_steps_per_second': 3.815,
 'epoch': 3.0}

In [26]:
trainer.save_model('t5-GEC-128len-9e')

Saving model checkpoint to t5-GEC-128len-9e
Configuration saved in t5-GEC-128len-9e/config.json
Configuration saved in t5-GEC-128len-9e/generation_config.json
Model weights saved in t5-GEC-128len-9e/pytorch_model.bin
tokenizer config file saved in t5-GEC-128len-9e/tokenizer_config.json
Special tokens file saved in t5-GEC-128len-9e/special_tokens_map.json
Copy vocab file to t5-GEC-128len-9e/spiece.model


In [27]:
!tar -zcvf t5-GEC-128len-9e.tar.gz /kaggle/working/t5-GEC-128len-9e

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
tar: Removing leading `/' from member names
/kaggle/working/t5-GEC-128len-9e/
/kaggle/working/t5-GEC-128len-9e/training_args.bin
/kaggle/working/t5-GEC-128len-9e/spiece.model
/kaggle/working/t5-GEC-128len-9e/special_tokens_map.json
/kaggle/working/t5-GEC-128len-9e/generation_config.json
/kaggle/working/t5-GEC-128len-9e/config.json
/kaggle/working/t5-GEC-128len-9e/tokenizer_config.json
/kaggle/working/t5-GEC-128len-9e/tokenizer.json
/kaggle/working/t5-GEC-128len-9e/pytorch_model.bin


In [28]:
%cd /kaggle/working
from IPython.display import FileLink
FileLink('t5-GEC-128len-9e.tar.gz')

/kaggle/working


/kaggle/working/t5-GEC-128len-9e.tar.gz